# Running *Molecular Dynamics* on AQE for Polymer Melt Simulation

This notebook will demonstrate the pipeline to simulate a polymer melt or a blend of polymers in a melt using GROMACS molecular dynamics software.\
A polymer melt refers to a state where a polymer is heated until it becomes a viscous, liquid-like fluid. This is often done to process the polymer into various shapes and forms through methods like injection molding or extrusion.\
A mixture of polymers in a melt means combining two or more different polymers while they are in this liquid state to create a new material with unique properties. This is similar to how metal alloys are made by mixing different metals.\
The pipeline will enable us to do the following:
- [x] Construction of _homopolymer_ with a user defined polymer chain length or molecular weight.
- [x] Construction of _homopolymer melt_ or _homopolymer blend_.
- [x] Computation of various thermodynamic properties of the melt and polymeric properties.

A homopolymer is a polymer that is made up of only one type of monomer. Essentially, it consists of repeating units of the same molecule. For example, polyethylene is a homopolymer made entirely from ethylene monomers.
Construction of homopolymer means creating a polymer that consists of just one type of monomer, either with a specific chain length or molecular weight. This is in contrast to a copolymer, which would be made from two or more different monomers.

**The objective of this notebook is to enable user to compute:**
- [x] **The glass transition temperature (Tg) of the polymer/polymer mixture:** The glass transition temperature is a critical property in the study of polymers. It refers to the temperature range at which a polymer transitions from a hard, glassy material to a soft, rubbery state. Below the glass transition temperature, the polymer chains are in a rigid and brittle state, but above this temperature, they gain mobility and become more flexible and pliable.
- [x] **Cohesive energy density (CED) of polymer:** Cohesive energy density of a polymer is a measure of the energy required to separate the polymer molecules from each other to an infinite distance. It's essentially a way of quantifying the intermolecular forces within the polymer. In other words, CED gives an idea of how strongly the polymer molecules are attracted to each other. A higher cohesive energy density means the molecules are more strongly bound together, which can influence the material's properties like its solubility, mechanical strength, and glass transition temperature.
- [x] **Polymeric structural properties:** End-to-end distance distribution, radius of gyration and its eigen values.
- [x] **Diffusion coefficient of polymer:** The diffusion coefficient of a polymer is a measure of how quickly polymer molecules move or spread through a medium (like a solvent or another polymer). It indicates the rate at which individual polymer chains travel within the given material. It's important in various applications, such as in Material Science when studying the mobility of polymers and their response to environmental changes, or Polymer Processing when understanding how polymers mix and spread during manufacturing.

The major steps involved in the pipeline is as follows:
- **Step 1:** Load AiiDA profile.
- **Step 2:** Import necessary python packages.
- **Step 3:** Setup computer and code.
- **Step 4:** User inputs for running the molecular dynamics simulation.
- **Step 5:** Polymer construction from the monomer.
- **Step 6:** Setup for molecular dynamics simulation. (i.e. - energy minimization, equilibration simulation, production simulation)
- **Step 7:** Running the equilibration molecular dynamics simulation.
- **Step 8:** Calculation of equilibrium properties.
- **Step 9:** Visualization of properties and polymer melt system.
- **Step 10:** Setup for molecular dynamics simulation for multiple thermodynamic conditions required for some properties calculation.
- **Step 11:** Running MD simulation at multiple thermodynamic conditions.
- **Step 12:** Calculation of polymer properties abd visualization.

We will go through these aformentioned steps step by step.

![PoC](flow1.svg)

<div style="background-color:yellow; padding:5px; font-size:18px;">
    <strong>Section A:</strong> AiiDA profile, HPC info and Simulation Setup.
</div>

**Step 1:** Load AiiDA profile.\
We load an AiiDA profile. The `load_profile` method will load the default profile if no explicit profile name is specified.

In [9]:
from aiida import load_profile
load_profile()

Profile<uuid='e3eed8e7f41e49b094d7e9107f76a58a' name='gromacs'>

**Step 2:** Import necessary python packages.

In [14]:
# Import basic packages
import pathlib
import os
import numpy as np

# Import aiida packages
from aiida.engine import run, submit
from aiida.engine import calcfunction, WorkChain
from aiida.orm import Bool, Int, Float, Str, Dict, List, SinglefileData, FolderData, Computer, load_code, load_node
from aiida_shell import launch_shell_job

# Import plotting packages
from IPython.display import display, Image

# Import custom packages
import utils.polymer_constant
from utils.polymerize import PolymerizeWorkChain
import utils.gromacs_setup
import utils.gromacs_analysis

**Step 3:** Setup computer and code.

As we are proceeding towards the MD simulation, we must ensure that the simulations are running on the HPC to maximize the performance. Therefore, we are loading the AiiDA variable `Computer` where the information about the HPC will be given to submit the simulation at HPC.

In [15]:
# Computer name
computer_name = 'ondemand-bd08a50fcb1d2f09'

# GROMACS code
code_name = 'gmx-gpu'
gpu = False
if 'gpu' in code_name:
    gpu = True

# Partition name
partition_name = 'spc36c1g'
 
computer = Computer.collection.get(label=computer_name)
print(f'Computer name:\t\t\t{computer.label}')

if '@' in code_name:
    gmx_code = load_code(label=f'{code_name}')
else:
    gmx_code = load_code(label=f'{code_name}@{computer.label}')
gmx_local = load_code(label='gmx@localhost')

print(f'You are using code:\t{gmx_code.label}@{computer.label}')
print(f'GPU status of code:\t{gpu}')
print(f'Local gromacs code:\tgmx@localhost (Used for all the steps except running simulation)')
if computer_name != 'localhost':
    print(f'Simulations will run on partition:\t{partition_name}')
else:
    print(f'Simulations will run on partition:\tlocalhost')

Computer name:			ondemand-bd08a50fcb1d2f09
You are using code:	gmx-gpu@ondemand-bd08a50fcb1d2f09
GPU status of code:	True
Local gromacs code:	gmx@localhost (Used for all the steps except running simulation)
Simulations will run on partition:	spc36c1g


Also, we need to setup the SLURM job submission parameters setup for submitting GROMACS job in specific node. E.g. - Here we are using 32 cores at the PARTITION prm96c4g.

In [16]:
sch_comm = ""
if computer_name != 'localhost':
    sch_comm += f"#SBATCH --partition={partition_name}"
prep_text = """
        module load gromacs
        """
if gpu:
    sch_comm += "\n#SBATCH --gpus=1"
    prep_text = """
            module load cuda
            module load gromacs/cuda-mpi
            export OMP_NUM_THREADS=32
            export GMX_ENABLE_DIRECT_GPU_COMM=1
            #export GMX_GPU_DD_COMMS=1
            #export GMX_GPU_PME_PP=1
            #export GMX_FORCE_UPDATE_DEFAULT_GPU=1
            """

metadata = {
    "options": {
        "computer": computer,
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 1,
        },
        "max_wallclock_seconds": 7200,  # 2 hours
        "custom_scheduler_commands": sch_comm,
        "prepend_text": prep_text
    }
}

**Step 3:** User inputs for running the molecular dynamics simulation.

Current PoC has 5 polymers. The complete table of polymers with their `monomer_id` is given below. The `monomer_id` will be the input for MD simulations.

|    Polymer Name      | Monomer ID <br> (`monomer_id`) |
|:--------------------:|:------------:|
| t-Poly butadiene     |      1       |
| Poly-vinyl alcohol   |      2       |
| Poly-methyl acrylate |      3       |
| c-isoprene           |      4       |
| polystyrene          |      5       |

We define a few **_user defined_** parameters which are used throughout the pipeline. The necessary parameters are classified in two group of parameters.

- **_Parameters for polymer/monomer_**
1. **`monomer_id`:** Id of the polymer for the MD simulation.
2. **`monomer_count_per_polymer`:** Number of monomer units per polymer.
3. **`polymer_count`:** Number of polymer units for MD simulations.

- **_MD properties_**
1. **`property_list`:** List of properties to be calculated from the MD simulation.

- **_Parameters for MD simulations (Thermodynamic conditions)_**
1. **`temperature`:** Simulation temperature in Kelvin.
2. **`pressure`:** Simulation pressure in bar.
3. **`dt`:** Simulation timestep in ps.
4. **`nsteps`:** Number of MD steps.

- **_Parameters for MD simulations (Specific for calculation of some properties)_**
1. **`search_region_Tg`:** Simulation temperature range is given for the properties calculation.
2. **`n_temperature`:** Number of simulation temperature is given for the properties calculation.

In [17]:
# User inputs

# Monomer inputs
monomer_id = 2                                       # id list will be shown in the terminal
monomer_count_per_polymer = 100                      # number of monomers in one polymer
polymer_count = 50                                   # number of polymers to be simulated

# MD inputs (primary)
temperature = 380.15                                 # simulation temperature (K)
pressure = 1.0                                       # simulation pressure (bar)
property_list = ['Potential', 'Density', 'Tg']       # properties to be calculated
dt = 0.002                                           # timestep (in ps)
nsteps = 50000                                       # md steps (# of steps)
t_sample = 10000                                     # last sample time (in ps)

'''
    This will be a seperate cell only asked for user input if there is a requirement.
'''
# Optional parameters for property calculations
# For Tg
search_region_Tg = [335, 425]                        # user expected Tg
n_temperature = 10                                   # number of temperature points
nsteps_tg = 10000                                    # number of md steps for tg (# of steps)

Convert the user input into `aiida.orm` data types accepted by the `node` arguments. 

In [18]:
monomer_id = Int(monomer_id)
monomer_pdbfname = utils.polymer_constant.get_pdb(monomer_id)
monomer_count_per_polymer = Int(monomer_count_per_polymer)

polymer_connection_point_list = utils.polymer_constant.get_connection_point(monomer_id)
polymer_count = Int(polymer_count)

temperature = Float(temperature)
pressure = Float(pressure)
cls_prop_list = utils.polymer_constant.get_classified_property_list(List(property_list))
primary_property_list = List(cls_prop_list[0])
secondary_property_list = List(cls_prop_list[1])
dt = Float(dt)
nsteps = Int(nsteps)
t_sample_start = dt.value * nsteps.value - t_sample
t_sample_start = Float(t_sample_start)

#temperature_list = List([])
if secondary_property_list:
    for prop in secondary_property_list.get_list():
        if prop == 'Tg':
            if search_region_Tg[0] > search_region_Tg[1]:
                raise ValueError('Temperature range is not correct.')
            temperature_list = List(np.linspace(search_region_Tg[0], search_region_Tg[1], n_temperature).tolist())
    nsteps_tg = Int(nsteps_tg)
    t_sample_start_tg = dt.value * nsteps_tg.value - t_sample
    t_sample_star_tg = Float(t_sample_start_tg)

**Step 5:** Polymer construction from the monomer.

The monomer pdb file is opened and wrapped it in AiiDA's `SinglefieData` data type, which allows it to be stored in the provenance graph.

In [19]:
monomer = SinglefileData(os.getcwd() + '/monomer_data/' + monomer_pdbfname.value)

We use the PDB file containing the monomer structure and use a run(polymerize) workchain to automate the polymerization process. This workchain identifies the bonding sites in the monomer, iteratively connects them to form a polymer, and run till the number of monomer per chain is achieved. The output of this step is a dictionary containing the polymer's structural data wrapped in AiiDA's `SinglefileData` data type and atomic weight of the polymer chain. This streamlined approach eliminates the need for manual intervention.

<div style="background-color:yellow; padding:5px; font-size:18px;">
    <strong>Section B:</strong> Polymerization and MD setup.
</div>

In [20]:
polymer_data = run(PolymerizeWorkChain, monomer = monomer, monomer_count = monomer_count_per_polymer, polymer_connection_point_list = polymer_connection_point_list)

Polymerization starts for PVAMonomer.pdb ->
100


Visualization of Polymer Chain

In [21]:
import nglview as nv
import tempfile

def visualize(node: SinglefileData) -> nv.NGLWidget:
    with node.open(mode='rb') as source:
        with tempfile.NamedTemporaryFile(suffix=node.filename) as target:
            target.write(source.read())
            view = nv.NGLWidget()
            view.add_component(target.name)
            #view.clear_representations()
            view.add_ball_and_stick()
            return view

In [22]:
visualize(polymer_data['polymer'])

NGLWidget()

**Step 6:** Setup for molecular dynamics simulation.\
We configure the force-field folder and encapsulate it within AiiDA's `FolderData` data type, enabling its storage in the provenance graph. This folder contains the definitions for nonbonded and bonded interactions specific to the chosen force field, which will be utilized throughout the subsequent steps of the workflow.

In [23]:
# Default Parameters
ffdir = os.getcwd() + '/oplsaa.ff'
oplsaa = FolderData(tree=ffdir)

`gmx pdb2gmx` is used to generate the forcefield information for the given polymer i.e.  nonbonded and bonded interaction parameters for all the atoms in the polymer

The `gmx pdb2gmx` is expected to generate three outputs of interest:
- `polymer_out.pdb`: `.pdb` file of the polymer.
- `topol.top`: The topology file.
- `posre.itp`: The position restraint file included in the topology file.

In [24]:
# Run `gmx pdb2gmx` to convert the PDB to GROMACS .gro format.
results_pdb2gmx, node_pdb2gmx = launch_shell_job(
    gmx_local,
    arguments='pdb2gmx -f {polymer} -o polymer_out.pdb -water spce -ff oplsaa',
    nodes={
        'polymer': polymer_data['polymer'],
        'folder': oplsaa,
    },
    filenames={
        'folder': 'oplsaa.ff',
    },
    outputs=['polymer_out.pdb', 'topol.top', 'posre.itp'],
    metadata={'call_link_label': 'polymerize', 'options': {'redirect_stderr': True}},
)
#print(results_pdb2gmx['stdout'].get_content())

A single chain will be equilibrated here in 2 steps here.
1. Energy minimization of single chain.
2. 100 ps simulation of single chain to equilibrate.

This step will optimize the structure of the chain ensuring the insertion of polymer chains in the simulation box.

**_i. Energy minimization of single chain._**

In [25]:
# Equilibration of single chain
results_sc_editconf, node_editconf_sc = launch_shell_job(
    gmx_local,
    arguments='editconf -f {gro} -d 1.0 -o polymer_out_box.pdb',
    nodes={
        'gro': results_pdb2gmx['polymer_out_pdb'],
    },
    outputs=['polymer_out_box.pdb'],
    metadata={'call_link_label': 'pdb2gmx', 'options': {'redirect_stderr': True}},
)

# Run `gmx grompp` to pre-process the parameters for energy minimization.
results_grompp_sc_em, node_grompp_sc_em = launch_shell_job(
    gmx_local,
    arguments='grompp -f {mdp} -c {gro} -p {top} -o em_sc.tpr',
    nodes={
        'mdp': utils.gromacs_setup.get_em_mdp(),
        'gro': results_sc_editconf['polymer_out_box_pdb'],
        'top': results_pdb2gmx['topol_top'],
        'posre': results_pdb2gmx['posre_itp'],
        'folder': oplsaa,
    },
    filenames={
        'folder': 'oplsaa.ff'
    },
    outputs=['em_sc.tpr'],
    metadata={'call_link_label': 'pdb2gmx', 'options': {'redirect_stderr': True}},
)
#print(results_grompp_sc_em['stdout'].get_content())

# Run `gmx mdrun` to run the energy minimization.
results_sc_em, node_sc_em = launch_shell_job(
    gmx_local,
    arguments='mdrun -v -deffnm em_sc -s {tpr}',
    nodes={
        'tpr': results_grompp_sc_em['em_sc_tpr'],
    },
    outputs=['em_sc.*'],
    metadata={'call_link_label': 'em_sc_grompp', 'options': {'redirect_stderr': True}},
)
#print(results_sc_em['stdout'].get_content())

**_ii. 100 ps simulation of single chain to equilibrate._**

In [26]:
# Run `gmx grompp` to pre-process the parameters for equilibration.
results_grompp_sc_eqnvt, node_grompp_sc_eqnvt = launch_shell_job(
    gmx_local,
    arguments='grompp -f {mdp} -c {gro} -p {top} -o eqnvt_sc.tpr',
    nodes={
        'mdp': utils.gromacs_setup.get_nvt_mdp(temperature = temperature),
        'gro': results_sc_em['em_sc_gro'],
        'top': results_pdb2gmx['topol_top'],
        'posre': results_pdb2gmx['posre_itp'],
        'folder': oplsaa,
    },
    filenames={
        'folder': 'oplsaa.ff'
    },
    outputs=['eqnvt_sc.tpr'],
    metadata={'call_link_label': 'em_sc', 'options': {'redirect_stderr': True}},
)
#print(results_grompp_sc_em['stdout'].get_content())

# Run `gmx mdrun` to run the equilibration of single chain.
arguments = 'mdrun -v -deffnm eqnvt_sc -s {tpr}'
if gpu:
    arguments += ' -update gpu -bonded gpu -pme gpu -pmefft gpu -nb gpu'
results_sc_eqnvt, node_sc_eqvpt = launch_shell_job(
    gmx_code,
    arguments=arguments,
    nodes={
        'tpr': results_grompp_sc_eqnvt['eqnvt_sc_tpr'],
    },
    outputs=['eqnvt_sc.*'],
    metadata=metadata,
)
#print(results_sc_eqnvt['stdout'].get_content())

The simulation box length is determined using the `calc_simulation_box_length` function. This calculation is based on the default polymer melt density of 0.40 g/cc and the specified number of polymer chains for the simulation. By leveraging this density, the function ensures that the simulation box is appropriately sized to represent the polymer melt environment accurately, maintaining realistic molecular interactions and spatial arrangements. This calculated box length serves as the foundation for subsequent simulation setup and execution.

In [27]:
box_length = utils.gromacs_setup.calc_simulation_box_length(polymer_data['polymer_molecular_weight'], polymer_count)

Similar to the previous steps, GROMACS creates a simulation box where mutiple chains of the polymer will be inserted for the MD simulations. From the previous step, `.pdb` file of the single polymer chain is used to replicate the polymer chain inside the simulation box. In the output, we get a simulation box containing `polymer_count` number of polymers.

In [28]:
run = Bool(True)
iter = 0
while run.value:
    results_insert, node_insert = launch_shell_job(
        gmx_local,
        arguments='insert-molecules -box {box_length} -ci {polymer} -nmol {polymer_count} -try 999 -o melt.pdb',
        nodes={
            'box_length': box_length,
            'polymer': results_sc_eqnvt['eqnvt_sc_gro'],
            'polymer_count': polymer_count,
        },
        outputs=['melt.pdb'],
        metadata={'call_link_label': 'polymerize', 'options': {'redirect_stderr': True}},
    )
    #print(results_insert['stdout'].get_content())
    polymer_count_inserted = utils.gromacs_setup.check_insert_molecules(results_insert['stdout'], polymer_count)
    if polymer_count_inserted.value != polymer_count.value:
        run = Bool(True)
        box_length = Float(box_length + 5.0)
        iter += 1
        if iter > 100:
            polymer_count = Int(polymer_count_inserted.value)
            print(f'WARNING: {polymer_count_inserted.value} polymers are inserted in the simulation box.')
            run = Bool(False)
    else:
        run = Bool(False)

In [29]:
visualize(results_insert['melt_pdb'])

NGLWidget()

While running the `pdb2gmx` command we have created the topology for one single polymer chain. However, in the `topol.top` file we have to provide the number of polymer chain present in the simulation box. Therefore, we are updating the number of molecules line from 1 to `polymer_count` using the `sed` command.

In [47]:
results_updatetop, node_updatetop = launch_shell_job(
    'sed',
    arguments='-i "s/Other               1/Other               {polymer_count}/g" {top}',
    nodes={
        'polymer_count': polymer_count,
        'top': results_pdb2gmx['topol_top'],
    },
    outputs=['topol.top'],
    metadata={'call_link_label': 'pdb2gmx', 'options': {'redirect_stderr': True}},
)
#print(results_updatetop['stdout'].get_content())

<div style="background-color:yellow; padding:5px; font-size:18px;">
    <strong>Section C:</strong> MD Simulation Pipeline.
</div>

**Prerequisite for MD simulations and simulation box preparation is completed and we can proceed to run the molecular dynamics simulation.**

`gmx grompp` is used to generate a `.tpr` file written in binary format which has all the necessary input information for running the MD simulation.

The `gmx grompp` takes the input of three necesary files to generate the `.tpr` file.
- [x] `.mdp`: The molecular dynamics parameters.
- [x] `.gro`: The coordinate information of all the polymer chains present in the simulation box.
- [x] `.top`: The tolology file of the polymer. Also, this has the information of number of polymers present in the simulation box.

In [48]:
# Run `gmx grompp` to pre-process the parameters for energy minimization.
results_grompp_em, node_grompp_em = launch_shell_job(
    gmx_local,
    arguments='grompp -f {mdp} -c {gro} -p {top} -o em.tpr',
    nodes={
        'mdp': utils.gromacs_setup.get_em_mdp(),
        'gro': results_insert['melt_pdb'],
        'top': results_updatetop['topol_top'],
        'folder': oplsaa,
    },
    filenames={
        'folder': 'oplsaa.ff'
    },
    outputs=['em.tpr'],
    metadata={'call_link_label': 'updatetop', 'options': {'redirect_stderr': True}},
)
#print(results_grompp_em['stdout'].get_content())

Now, the generated `.tpr` file is used for running the molecular dynamics simulation. We can run the `gmx mdrun` to run the energy minimization step. The variable `computer` is used in the `launch_shell_job` command ensures the use of HPC while running the simulation.

In [49]:
# Run `gmx mdrun` to run the energy minimization.
results_em, node_em = launch_shell_job(
    gmx_code,
    arguments='mdrun -v -deffnm em -s {tpr}',
    nodes={
        'tpr': results_grompp_em['em_tpr'],
    },
    outputs=['em.*'],
    metadata=metadata,
)
#print(results_em['stdout'].get_content())

Similar to the previous `gmx grompp` step, we can proceed with NPT equilibration step where the molecular dynamics simulation will be performed at the given thermodynamic conditions given by the user.

In [50]:
if primary_property_list:
    npt_mdp = utils.gromacs_setup.get_npt_mdp(temperature = temperature, pressure = pressure, dt = dt, nsteps = nsteps)
    
    # Run `gmx grompp` to pre-process the parameters for equilibrium NPT simulation.
    results_grompp_eqnpt, node_grompp_eqnpt = launch_shell_job(
        gmx_local,
        arguments='grompp -f {mdp} -c {gro} -p {top} -o npt.tpr',
        nodes={
            'mdp': npt_mdp,
            'gro': results_em['em_gro'],
            'top': results_updatetop['topol_top'],
            'folder': oplsaa,
        },
        filenames={
            'folder': 'oplsaa.ff'
        },
        outputs=['npt.tpr'],
        metadata={'call_link_label': 'em_mdrun', 'options': {'redirect_stderr': True}},
    )
    #print(results_grompp_eqnpt['stdout'].get_content())

**Step 7:** Running the equilibration molecular dynamics simulation.
The generated `.tpr` file is used for running the molecular dynamics simulation for NPT equilibration. We can run the `gmx mdrun` to run the NPT equilibration step.

In [51]:
if primary_property_list:
    arguments = 'mdrun -v -deffnm npt -s {tpr}'
    if gpu:
        arguments += ' -update gpu -bonded gpu -pme gpu -pmefft gpu -nb gpu'
    # Run `gmx mdrun` to run the equilibrium NPT simulation
    results_eqnpt, node_eqnpt = launch_shell_job(
        gmx_code,
        arguments=arguments,
        nodes={
            'tpr': results_grompp_eqnpt['npt_tpr'],
        },
        outputs=['npt.*'],
        metadata=metadata,
    )
    #print(results_eqnpt['stdout'].get_content())

<div style="background-color:yellow; padding:5px; font-size:18px;">
    <strong>Section D:</strong> Property Calculation.
</div>

**Step 8:** Calculation of equilibrium properties.

In [8]:
if primary_property_list:
    primary_property_str = ''
    for iproperty in primary_property_list.get_list():
        primary_property_str += iproperty + '\n'
    primary_property_str += '0'
    
    # Run `gmx energy` to extract the potential energy during the equilibrium NPT simulation
    results_energy, node_energy = launch_shell_job(
        gmx_local,
        arguments='energy -f {edr} -o energy.xvg',
        nodes={
            'edr': results_eqnpt['npt_edr'],
            'stdin': SinglefileData.from_string(primary_property_str),
        },
        outputs=['energy.xvg'],
        metadata={'options': {'redirect_stderr': True, 'filename_stdin': 'stdin'}},
    )
    
    average_eq_property_list = utils.gromacs_analysis.get_average_property(results_energy['stdout'], primary_property_list)
    print(average_eq_property_list)
    print('Average equilibrium properties:\n')
    for average_property in average_eq_property_list:
        print(f'{average_property[0]}: {average_property[1]} {average_property[2]}\n')

uuid: 886f7d17-1239-4231-a71a-ab661a967673 (pk: 203855) value: [['Potential', '311110', '(kJ/mol)'], ['Density', '152.669', '(kg/m^3)']]

Average equilibrium properties:

Potential: 172000 (kJ/mol)

Density: 1162.669 (kg/m^3))


**Step 9:** Visualization of properties and polymer melt system.

To visualize the data, first we need to import necessary python packages to be installed.

We import the `gromacs_analysis` to create the plot for properties calculated in the previous step.

In [ ]:
if primary_property_list:
    plot = utils.gromacs_analysis.create_time_plot(results_energy['energy_xvg'])
    for iplot in plot:
        display(Image(filename=iplot))

![polt0](time_plot_0.png)

![polt1](time_plot_1.png)

<div style="background-color:yellow; padding:5px; font-size:18px;">
    <strong>Section C (Repeat):</strong> MD Simulation Pipeline.
</div>

**Step 10:** Setup for molecular dynamics simulation for multiple thermodynamic conditions required for some properties calculation.

We are going to perform multiple molecular dynamics simulation by varying temperatures or varying pressure or varying both temperature and pressure. This variation in temeprature and pressure is controlled based on the user input.

As the varying parameters are temperature and pressure, we do not need to perform the complete gromacs setup as we performed previously. We are executing the GROMACS `grompp` command to generate `.tpr` file for all the thermodynamics conditions (i.e. - temperature range or pressure range or both.) These `.tpr` files will be used to run the molecular dynamics simulations. 

In [54]:
if secondary_property_list:
    node_grompplist = List([])
    results_grompplist = List([])
    
    for i, temperature_i in enumerate(temperature_list.get_list()):
        id = Int(i)
        temperature_curr = Float(temperature_i)
        eqnpt_mdp = utils.gromacs_setup.get_npt_mdp(id = id, temperature = temperature_curr, pressure = pressure, dt = dt, nsteps = nsteps)
        
        tpr = f'npt-{i}.tpr'
        results_grompp_eqnpt_iter, node_grompp_eqnpt_iter = launch_shell_job(
            gmx_local,
            arguments='grompp -f {mdp} -c {gro} -p {top} -o {tpr}',
            nodes={
                'mdp': eqnpt_mdp,
                'gro': results_em['em_gro'],
                'top': results_updatetop['topol_top'],
                'tpr': Str(tpr),
                'folder': oplsaa,
            },
            filenames={
                'folder': 'oplsaa.ff'
            },
            outputs=[tpr],
            metadata={'options': {'redirect_stderr': True}},
        )
        node_grompplist.append(node_grompp_eqnpt_iter)
        results_grompplist.append(results_grompp_eqnpt_iter)

**Step 11:** Running MD simulation at multiple thermodynamic conditions.

The series of molecular dynamics simulation is running. A `list` of ouput will be generated for different thermodynamic conditions.

In [55]:
if secondary_property_list:
    # Run `gmx mdrun` to run the equilibrium NPT simulations at different T & P
    node_eqnptlist = List([])
    results_eqnptlist = List([])

    arguments = 'mdrun -v -deffnm {output} -s {tpr}'
    if gpu:
        arguments += ' -update gpu -bonded gpu -pme gpu -pmefft gpu -nb gpu'
    
    for i, temperature_ele in enumerate(temperature_list.get_list()):
        #print(i, temperature_ele, type(temperature_ele), type(i))
        tpr = f'npt_{i}_tpr'
        output = f'npt_{i}'
        results_eqnpt, node_eqnpt = launch_shell_job(
            gmx_code,
            arguments=arguments,
            nodes={
                'tpr': results_grompplist[i][tpr],
                'output': Str(output),
            },
            outputs=[f'{output}.*'],
            metadata=metadata,
        )
        node_eqnptlist.append(node_eqnpt)
        results_eqnptlist.append(results_eqnpt)

<div style="background-color:yellow; padding:5px; font-size:18px;">
    <strong>Section D (Repeat):</strong> Polymeric Property Calculation.
</div>

**Step 12:** Calculation of polymer properties and visualization.

Secondary properties for this system at all the thermodynamic condtions are getting calculated.

In [84]:
if secondary_property_list:
    gromacs_property_list = utils.polymer_constant.get_all_gromacs_property_list(secondary_property_list)
    secondary_property_str = ''
    for iproperty in gromacs_property_list.get_list():
        secondary_property_str += iproperty + '\n'
    secondary_property_str += '0'
    
    node_energylist = List([])
    results_energylist = List([])
    average_property_list = List([])
    
    for i, temperature_ele in enumerate(temperature_list.get_list()):
        edr = f'npt_{i}_edr'
        output = f'energy_{i}.xvg'
        results_energy, node_energy = launch_shell_job(
            gmx_local,
            arguments='energy -f {edr} -o {output} -b {t_sample_start_tg}',
            nodes={
                'edr': results_eqnptlist[i][edr],
                'output': Str(output),
                't_sample_start_tg': t_sample_start_tg,
                'stdin': SinglefileData.from_string(secondary_property_str),
            },
            outputs=[f'{output}'],
            metadata={'options': {'redirect_stderr': True, 'filename_stdin': 'stdin'}},
        )
        node_energylist.append(node_energy)
        results_energylist.append(results_energy)

        average_property_list.append(utils.gromacs_analysis.calc_average_property(results_energy[f'energy_{i}_xvg']))

We call the `gromacs_analysis` to create the plot for properties calculated in the previous step.

In [91]:
if secondary_property_list:
    for iproperty in secondary_property_list:        

        if iproperty == 'Tg':
            icol = -1
            for i, average_property in enumerate(average_property_list):
                if average_property[i][3] == 'Density':
                    icol = i
                    break
            if icol < 0:
                raise ValueError('ERROR: Property not calculated.')
            plot = utils.gromacs_analysis.create_tg_plot(temperature_list, average_property_list, Int(icol))

            print(f'Plotting data for calculating: {iproperty}')
            
            display(Image(filename=plot.value))

Plotting data for calculating: Tg


![Tg](density_temperature_plot_with_vertical_Tg1.png)